In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

# import requests
# import scrapy
# import time
import pandas as pd
from pprint import pprint
# import re
# import json
# import datetime

from scrapers import staff

In [2]:
data = staff.main()

Fri Sep 17 20:21:36 2021
Starting crawling 'https://staff.am'.
1515 jobs detected.
Starting crawling job postings..
https://staff.am/en/php-backend-developer
1/1515 Est.: 252.33m.
https://staff.am/en/tumo-labs-application-developer-6
2/1515 Est.: 252.17m.
https://staff.am/en/scrum-master-192
3/1515 Est.: 252.0m.
https://staff.am/en/logistics-manager-50
4/1515 Est.: 251.83m.
https://staff.am/en/tumo-labs-full-stack-developer-6
5/1515 Est.: 251.67m.
https://staff.am/en/midsenior-level-php-software-engineer-24
6/1515 Est.: 251.5m.
https://staff.am/en/uiux-designer-719
7/1515 Est.: 251.33m.
https://staff.am/en/head-of-engineering-9
8/1515 Est.: 251.17m.
https://staff.am/en/photographer-14
9/1515 Est.: 251.0m.
https://staff.am/en/graphic-designer-703
10/1515 Est.: 250.83m.
https://staff.am/en/operations-monitoring-engineer-cloud-team-4
11/1515 Est.: 250.67m.
https://staff.am/en/senior-software-engineer-213
12/1515 Est.: 250.5m.
https://staff.am/en/administrative-assistant-285
13/1515 Est.: 